# ADM Homework 5

## 1 - Data

Let's take a first look at the data.

We have three files describing links in graphs, with their timestamps.

```
micf@MacBook-Pro-di-Michele ADM % wc -l ./ADM_HW5/datasets/sx-stackoverflow-a2q.txt
 17823525 ./ADM_HW5/datasets/sx-stackoverflow-a2q.txt
micf@MacBook-Pro-di-Michele ADM % wc -l ./ADM_HW5/datasets/sx-stackoverflow-c2a.txt
 25405374 ./ADM_HW5/datasets/sx-stackoverflow-c2a.txt
micf@MacBook-Pro-di-Michele ADM % wc -l ./ADM_HW5/datasets/sx-stackoverflow-c2q.txt
 20268151 ./ADM_HW5/datasets/sx-stackoverflow-c2q.txt
```

The total number of links is ~63M, which we'll need to merge in a single graph

```
micf@MacBook-Pro-di-Michele ADM % head ./ADM_HW5/datasets/sx-stackoverflow-c2q.txt
4550 4550 1220729190
242 184 1220733503
4213 4946 1220768149
91 91 1220768295
2658 1874 1220771891
4035 1874 1220773037
2257 4489 1220802041
577 577 1220834891
4489 4489 1220853536
828 2783 1220854143
```

The file is a CSV-like format, where spaces are used as separators. For each row, the user that commented/answered, the user who posted the question/answer being replied to, and the timestamp of the interaction (UNIX timestamp in seconds) are provided.

Since the files end with newlines, we can just append them together to get our final graph and do actual pre-processing on everything at once.

```
micf@MacBook-Pro-di-Michele ADM % cat ./ADM_HW5/datasets/sx-stackoverflow-c2q.txt ./ADM_HW5/datasets/sx-stackoverflow-c2a.txt ./ADM_HW5/datasets/sx-stackoverflow-a2q.txt > ./ADM_HW5/datasets/merged_graph.txt
micf@MacBook-Pro-di-Michele ADM % wc -l ./ADM_HW5/datasets/merged_graph.txt
 63497050 ./ADM_HW5/datasets/merged_graph.txt
```

Now, depending on the functionality, we can load one of the three sub-graphs, or the complete graph, as we deem fit.

## 2 & 3 - Functionalities

### Functionality 1

In [1]:
from functionality_utils import functionality_1

functionality_1()

,Value
Is Directed,True
# Users,1148384
# Links,15801794
Avg. Links per User,13.760026
Density,0.000012
Is Sparse,True


## Algorithmic Question

The problem asks to assign a set of students to one of dormitories so that given pairs of students are not in the same dormitory.

The problem can be modeled as a graph 2-coloring, where we build a graph such that:
- The nodes are the students
- An edge ($p_i$, $p_j$) exists if and only if students $p_i$ and $p_j$ have to be assigned to different dormitories.

Graph 2-coloring is an extension of traditional DFS/BFS (DFS is implemented here) and keeps its complexity of O(n+k), with n being the number of nodes in the graph and k the number of edges.

Here is an implementation of the solution:

In [2]:
from typing import Optional

def assign_students(num_nodes: int, edges: list[tuple[int, int]]) -> Optional[list[int]]:
    """
    Return a list where the i-th element is 0 if the i-th student is assigned to the first dormitory, 1 otherwise.

    If no possible assignment exist, this method returns None.

    :param num_nodes: Number of nodes in the graph (i.e. the number of students)
    :param edges: list of edges, given as pairs of student IDs
    :return: A list describing the assignment of students to dormitories, or None if an assignment is not possible.
    """
    colors = [-1]*num_nodes

    # Build adjacency lists
    adjacents = [[] for _ in range(num_nodes)]
    for i, j in edges:
        adjacents[i].append(j)
        adjacents[j].append(i)

    def visit_node(node: int, assign_color: int) -> bool:
        """
        Visit a node, coloring it and recursing on unvisited neighbors.

        :param node: index of the node to visit.
        :param assign_color: color to assign to this node.
        :return: True if the node and its neighbors got assigned a color, False if a conflict was detected.
        """
        colors[node] = assign_color
        neighbors_color = 1 if assign_color == 0 else 0
        for adjacent in adjacents[node]:
            if colors[adjacent] == -1:
                if not visit_node(adjacent, neighbors_color):
                    return False
            elif colors[adjacent] != neighbors_color:
                return False

        return True

    for i in range(len(colors)):
        if colors[i] == -1:
            ok = visit_node(i, 0)
            if not ok:
                return None

    return colors

Let's try the function on a couple examples:

In [3]:
def run_problem(num_nodes: int, edges: list[tuple[int, int]]) -> None:
    problem_result = assign_students(num_nodes, edges)
    if not problem_result:
        print("No possible assignment")
    else:
        print(problem_result)

In [4]:
# No conflicts
run_problem(10, [])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


All students were assigned to the same dormitory, which is fine as there are no conflicts

In [5]:
# One conflict
run_problem(10, [(0,1)])
run_problem(10, [(0,9)])

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


One conflict leads to one student being assigned to another dormitory.

In [6]:
# Three students that all hate each other
run_problem(10, [(0,1), (1,2), (0,2)])

No possible assignment


Since we have a loop of three nodes, 2-coloring is impossible

In [7]:
# Linear graph
run_problem(10, [(i,i+1) for i in range(9)])

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


Each student must be in a different dormitory than the ones right before and after themselves, so assignment values alternate between the two dormitories.